![alt text](logo.png)

# PyAmplitude Tutorial

...What is Pyamplitude ?

Amplitude its a python-based client designed to interact with the entire set of Amplitude Analytics services (Redshift, Export API, Dashboard Rest API and Behavioral Cohorts API) following the official Amplitude Documentation which can be found at:  

https://amplitude.zendesk.com/hc/en-us/categories/115000290488-Technical-Resources

General Organization: 

    class AmplitudeRestApi   -> A wrapper around each Rest API resources. 
    class RedshfitAmplitude  -> A sql connector for the Amplitude AWS-Redshift database. 
    class ExportApi          -> A wrapper around the entire Amplitude export 

Aditional Modules: 
    
    class ProjectHandler     -> Manage several apps & secret keys
    
Aditional Features: 

    - Calculate each query cost.
    - Manage your query rate limits in a ETL context.
    - Visualize the quentity of requests and your query limit status. 

## Install

sudo pip install pyamplitude

## How to use PyAmplitude ?

Let's start by importing the ProjectHandler and passing a project_name, api_key and api_secret key as parameters.

In [ ]:
from pyamplitude.projectshandler import ProjectsHandler

Lets instance the ProjectHanlder with our first app called 'BubbleWay'...

In [ ]:
bubbleConector = ProjectsHandler(project_name = 'BubbleWay', 
                                 api_key      = <'API-KEY'>, 
                                 api_secret   = <'API-SECRET'>)

Hint: Use the __repr__ method to check your actual instance when used with several apps. 

In [ ]:
print bubbleConector

project_name: BubbleWay | api_key: API-KEY | secret_key: API-SECRET

In [ ]:
from pyamplitude.amplituderestapi import AmplitudeRestApi

# Querying data from the Amplitude Analytics REST API Dashboard

Great ! So let's use the Amplitude Rest Api to query some useful data for later analysis... 

In [ ]:
apiconector = AmplitudeRestApi(project_handler = bubbleConector,
                               show_logs       = False,
                               log_query_cost = False)

### Creating and using segments

Segments are represented as JSON arrays, where each element is a JSON object corresponding to a filter condition. First import and initialize the Segment class and add each query filter. in these cases we will be creating two segment definitions and for later use.

first_segment_definition = Segment()

first_segment_definition.add_filter(prop='country',op='is',values=['argentina','brasil'])

second_segment_definition = Segment()

second_segment_definition.add_filter(prop='country',op='is',values=['argentina','paraguay'])

### Using the AmplitudeRestApi module. 

##### First Example:  Querying Active and New Users count

In [ ]:
data = apiconector.get_active_and_new_user_count(start    = '20170814',
                                                 end      = '20170825',
                                                 m        = 'active',
                                                 interval = 1,
                                                 segment_definitions = [first_segment_definition, 
                                                                        second_segment_definition],
                                                 group_by            = None) 

If all goes well... you should receive a JSON response such as: 

{
    "data": {
        "series": [ 
                    [18600,15294,14164,12945,12585,11797,10113,9523,8321,7873,9053,8109],
                    [3264,3423,3397,2984,2916,2827,2918,2934,1800,1560,1240,1100]
        ],
        "seriesMeta": ["Argentina", "Brasil"],
        "xValues": ["2017-08-14", "2017-08-15", "2017-08-16", "2017-08-17", "2017-08-18", "2017-08-19", "2017-08-20", 
                    "2017-08-21", "2017-08-22", "2017-08-23", "2017-08-24", "2017-08-25"]
    }
}

### Using plotly to plot available data

Let's use Plotly to plot 'active' and 'new' users we are querying: 

In [8]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


active_users_response = {"data": {"series": [[18600,15294,14164,12945,12585,11797,10113,9523,8321,7873,9053,8109],[3264,3423,3397,2984,2916,2827,2918,2934,1800,1560,1240,1100]],
"seriesMeta": ["Argentina", "Brasil"],"xValues": ["2017-08-14", "2017-08-15", "2017-08-16", "2017-08-17", "2017-08-18", "2017-08-19", "2017-08-20", "2017-08-21", "2017-08-22", "2017-08-23", "2017-08-24", "2017-08-25"]}}

new_users_response = {"data": {"series": [[860,152,14,129,1255,117,1014,952,8673,7887,15000,18000],[326,343,339,298,291,282,291,294,180,160,1400,1600]],
"seriesMeta": ["Argentina", "Brasil"],"xValues": ["2017-08-14", "2017-08-15", "2017-08-16", "2017-08-17", "2017-08-18", "2017-08-19", "2017-08-20", "2017-08-21", "2017-08-22", "2017-08-23", "2017-08-24", "2017-08-25"]}}

ar_dau = active_users_response['data']['series'][0]
br_dau = active_users_response['data']['series'][1]
dates  = active_users_response['data']['xValues']

new_ar = new_users_response['data']['series'][0]
new_br = new_users_response['data']['series'][1]


argentina_new_trace = go.Scatter(x = dates, y = ar_dau, name = 'DAU | Argentina', mode = 'lines+markers')
brasil_new_trace    = go.Scatter(x = dates, y = br_dau, name = 'DAU | Brasil',    mode = 'lines+markers') 
new_ar_trace        = go.Scatter(x = dates, y = new_ar, name = 'NEW | Argentina', mode = 'lines')
new_br_trace        = go.Scatter(x = dates, y = new_br, name = 'NEW | Brasil',    mode = 'lines')

data = [argentina_new_trace, brasil_new_trace, new_ar_trace, new_br_trace]

layout = dict(title = 'Amplitude Rest Api | Daily Active Users for Argentina & Brasil',
              xaxis = dict(title = 'Dates'),
              yaxis = dict(title = 'DAU'))

fig = dict(data=data, layout=layout)

iplot(fig)

Other Resources: Session Length Distribution, Average Session Length, Average Sessions per User, User Composition, Events, Events List, Event Segmentation, Funnel Analysis, Retention Analysis, User Activity, User Search, Real-time Active Users, Revenue Analysis, Revenue LTV, Annotations...

## Aditional options

#### Using  calculate_query_cost = True parameter

With PyAmplitude you can calculate each query cost very easily by checking the log_query_cost parameter. In an ETL context you may use this option not to exceed query limits. If you want to know more about how each query cost is being calculated, please read:

https://amplitude.zendesk.com/hc/en-us/articles/205469748-Dashboard-Rest-API-Export-Amplitude-Dashboard-Data#request-limits

#### Using the show_logs parameter...

# Fetching data from Amplitude Redshift

As a addition you can query data from Amplitude Redshift using the AmplitudeReshift class.

In [ ]:
from pyamplitude.amplituderestshift import AmplitudeRedshift

In [ ]:
ConnectionHandler = AmplitudeRedshift(host='',
                                      user='BubbleUser',
                                      port='5439',
                                      password=<yourpassword>,
                                      dbname='bubble_db',
                                      schema= 'app123098',
                                      table=<db_table>,
                                      show_logs= True)

In [ ]:
query = 'SELECT * FROM app123098.bubble_db LIMIT 10'

In [ ]:
ConnectionHandler.execute_query(query=query)

NOTE: AmplitudeRedshift has a serious of prebuild methods to fetch a list of users, query user events as well as new users, but you can also pass your specific query using the execute_query method.

## Authors

Marcos Manuel Muraro